In [40]:
#folder_path = '/users/irisz/downloads/2023_elic/croatian/croatian_manual_annotated_1008'
#save_path = '/users/irisz/downloads/2023_elic/croatian/croatian_finetune_data'

folder_path = '/users/irisz/downloads/2023_elic/croatian/croatian_manual_annotated_1008_overlap_removed'
save_path = '/users/irisz/downloads/2023_elic/croatian/croatian_finetune_data'

In [41]:
# get file name list - wav and textgrid

from os import listdir
from os.path import isfile, join
files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
files

['ckm004-2022-07-07-Cres_04_overlap_removed_15.Textgrid',
 'ckm002-2022-01-04-Crikvenica_04_overlap_removed_15.Textgrid',
 'ckm004-2022-07-07-Cres_01_overlap_removed_15.Textgrid',
 'ckm001-2022-01-16-Trviž_02_overlap_removed_15.Textgrid',
 'ckm002-2022-01-04-Crikvenica_01_overlap_removed_15.Textgrid',
 'ckm006-2023-05-28-Kostrena_06_overlap_removed_15.Textgrid',
 '.DS_Store',
 'ckm005-2022-11-26-Vrbnik_04_overlap_removed_15.Textgrid',
 'ckm006-2023-05-28-Kostrena_03_overlap_removed_15.Textgrid',
 'ckm005-2022-11-26-Vrbnik_01_overlap_removed_15.Textgrid',
 'ckm001-2022-01-16-Trviž_03_overlap_removed_15.Textgrid',
 'ckm002-2022-01-04-Crikvenica_05_overlap_removed_15.Textgrid',
 'ckm006-2023-05-28-Kostrena_02_overlap_removed_15.Textgrid',
 'ckm006-2023-05-28-Kostrena_07_overlap_removed_15.Textgrid',
 'ckm005-2022-11-26-Vrbnik_05_overlap_removed_15.Textgrid',
 'ckm006-2023-05-28-Kostrena_04_overlap_removed_15.Textgrid',
 'ckm005-2022-11-26-Vrbnik_06_overlap_removed_15.Textgrid',
 'ckm006

In [42]:
# subset of the files that 
file_names = []
for file in files:
    if file.startswith('c'):
        file_names.append(file.split('.')[0])
file_names = list(set(file_names))
file_names

['ckm004-2022-07-07-Cres_04_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_07_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_02_overlap_removed_15',
 'ckm001-2022-01-16-Trviž_02_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_04_overlap_removed_15',
 'ckm006-2023-05-28-Kostrena_05_overlap_removed_15',
 'ckm005-2022-11-26-Vrbnik_03_overlap_removed_15',
 'ckm004-2022-07-07-Cres_02_overlap_removed_15',
 'ckm001-2022-01-16-Trviž_04_overlap_removed_15',
 'ckm006-2023-05-28-Kostrena_02_overlap_removed_15',
 'ckm006-2023-05-28-Kostrena_03_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_03_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_08_overlap_removed_15',
 'ckm001-2022-01-16-Trviž_03_overlap_removed_15',
 'ckm005-2022-11-26-Vrbnik_06_overlap_removed_15',
 'ckm004-2022-07-07-Cres_03_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_06_overlap_removed_15',
 'ckm006-2023-05-28-Kostrena_07_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_05_overlap_removed_15',
 

In [4]:

# get original time info from textgrid

import re


def get_interval(time_list):
    interval_all = []
    interval_curr = 1
    for i in range(len(time_list)):
        if i == 0:
            interval_all.append(1)
        else:
            if time_list[i] > time_list[i - 1]:
                interval_all.append(interval_curr)
                continue
            else:
                interval_curr = interval_curr + 1
                interval_all.append(interval_curr)
    return interval_all


def get_info_from_textgrid(textgrid_file, file_name):
    #with open(textgrid_file, 'r') as file:
    try:    # file encoding may be different, try here 
        with open(textgrid_file, encoding = "utf-8") as file:
            lines = [line.rstrip() for line in file]
    except:
        with open(textgrid_file, encoding = "utf-16") as file:
            lines = [line.rstrip() for line in file]
   
    text_list = []
    xmin_list = []
    xmax_list = []
    interval_list = []
    for line in lines[14:]:  #informations needed begin on the 9th lines
        if 'text =' in line:
            line = line.split('"')[1]
            if line != "":
                text_list.append(line)
                #print (len(text_list), len(xmin_list), len(xmax_list), len(interval_list), line, interval_list[-2:])
            else:
                xmin_list = xmin_list[:-1]
                xmax_list = xmax_list[:-1]
                interval_list = interval_list[:-1]
        if 'xmin' in line:
            time_min = line.split('=')[1].strip(' ')
            xmin_list.append(time_min)
        if 'xmax' in line:
            time_max = line.split('=')[1].strip(' ')
            xmax_list.append(time_max)
        if 'intervals [' in line:
            interval = line.split('[')[1].strip(']:')
            interval_list.append(interval)
        if 'intervals: size' in line:
            xmin_list = xmin_list[:-1]
            xmax_list = xmax_list[:-1]
    file_name_list = [file_name] * len(text_list)
    
    xmin_list = [float(item) for item in xmin_list]
    xmax_list = [float(item) for item in xmax_list]
    interval_list = [int(item) for item in interval_list] # annotation chunk
    
    tier_list = get_interval(interval_list)
    return text_list, xmin_list, xmax_list, interval_list, file_name_list, tier_list
        

In [43]:
text_list = []
xmin_list = []
xmax_list = []
interval_list = []
file_name_list = []
tier_list = []

for file_name in file_names:
    print (file_name)
    textgrid_file = folder_path + '/' + file_name + '.TextGrid' # file path for textgrid
    file_info_list = get_info_from_textgrid(textgrid_file, file_name)
    text_list = text_list + file_info_list[0]
    xmin_list = xmin_list + file_info_list[1]
    xmax_list = xmax_list + file_info_list[2]
    interval_list = interval_list + file_info_list[3]
    file_name_list = file_name_list + file_info_list[4]
    tier_list = tier_list + file_info_list[5]

ckm004-2022-07-07-Cres_04_overlap_removed_15
ckm002-2022-01-04-Crikvenica_07_overlap_removed_15
ckm002-2022-01-04-Crikvenica_02_overlap_removed_15
ckm001-2022-01-16-Trviž_02_overlap_removed_15
ckm002-2022-01-04-Crikvenica_04_overlap_removed_15
ckm006-2023-05-28-Kostrena_05_overlap_removed_15
ckm005-2022-11-26-Vrbnik_03_overlap_removed_15
ckm004-2022-07-07-Cres_02_overlap_removed_15
ckm001-2022-01-16-Trviž_04_overlap_removed_15
ckm006-2023-05-28-Kostrena_02_overlap_removed_15
ckm006-2023-05-28-Kostrena_03_overlap_removed_15
ckm002-2022-01-04-Crikvenica_03_overlap_removed_15
ckm002-2022-01-04-Crikvenica_08_overlap_removed_15
ckm001-2022-01-16-Trviž_03_overlap_removed_15
ckm005-2022-11-26-Vrbnik_06_overlap_removed_15
ckm004-2022-07-07-Cres_03_overlap_removed_15
ckm002-2022-01-04-Crikvenica_06_overlap_removed_15
ckm006-2023-05-28-Kostrena_07_overlap_removed_15
ckm002-2022-01-04-Crikvenica_05_overlap_removed_15
ckm005-2022-11-26-Vrbnik_01_overlap_removed_15
ckm006-2023-05-28-Kostrena_04_

In [44]:
# save text and time info in dataframe

import pandas as pd

df = pd.DataFrame()
df['file'] = file_name_list
df['text'] = text_list
df['xmin'] = xmin_list
df['xmax'] = xmax_list
df['interval'] = interval_list
df['tier'] = tier_list

In [45]:
df

,file,text,xmin,xmax,interval,tier
0,ckm004-2022-07-07-Cres_04_overlap_removed_15,Na Grobniku se jako dobro čuva govor,0.000000,4.441976,1,1
1,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Va škole se puno čuva, i Katedra je jaka.",117.091133,120.544704,2,1
2,ckm004-2022-07-07-Cres_04_overlap_removed_15,"A puno se na Grobnik radilo z drvom, ča ne?",202.971944,206.757870,3,1
3,ckm004-2022-07-07-Cres_04_overlap_removed_15,A je bilo ča za mesopusno vrime?,339.817105,342.209639,4,1
4,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Mi, ja znan da kad san ja krenula va školu da ...",6.517170,12.770724,1,2
...,...,...,...,...,...,...
4503,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Tako da @ sad @ će njoj bit malo legje da se m...,581.117400,586.880631,162,2
4504,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,"Mlada, a toliko @ va poslu @ ni dobro.",587.446607,590.414025,163,2
4505,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Rabela bi kakovoga mladića nać i oženit se i @...,590.414025,596.167381,164,2
4506,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,To @ ni od toga koristi baš.,596.167381,598.298191,165,2


In [46]:
set(df['file'].tolist()) # all file covered

{'ckm001-2022-01-16-Trviž_01_overlap_removed_15',
 'ckm001-2022-01-16-Trviž_02_overlap_removed_15',
 'ckm001-2022-01-16-Trviž_03_overlap_removed_15',
 'ckm001-2022-01-16-Trviž_04_overlap_removed_15',
 'ckm001-2022-01-16-Trviž_05_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_01_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_02_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_03_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_04_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_05_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_06_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_07_overlap_removed_15',
 'ckm002-2022-01-04-Crikvenica_08_overlap_removed_15',
 'ckm004-2022-07-07-Cres_01_overlap_removed_15',
 'ckm004-2022-07-07-Cres_02_overlap_removed_15',
 'ckm004-2022-07-07-Cres_03_overlap_removed_15',
 'ckm004-2022-07-07-Cres_04_overlap_removed_15',
 'ckm005-2022-11-26-Vrbnik_01_overlap_removed_15',
 'ckm005-2022-11-26-Vrbnik_02_overlap_removed_15',
 'ckm00

In [9]:
# classify word or sentence level

def check_word(text):
    if '.' in text or ',' in text or ' ' in text or '?' in text or '!' in text:
        return 1
    else:
        return 0
    

In [47]:
df['sent'] = df['text'].apply(lambda x: check_word(x))
df

,file,text,xmin,xmax,interval,tier,sent
0,ckm004-2022-07-07-Cres_04_overlap_removed_15,Na Grobniku se jako dobro čuva govor,0.000000,4.441976,1,1,1
1,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Va škole se puno čuva, i Katedra je jaka.",117.091133,120.544704,2,1,1
2,ckm004-2022-07-07-Cres_04_overlap_removed_15,"A puno se na Grobnik radilo z drvom, ča ne?",202.971944,206.757870,3,1,1
3,ckm004-2022-07-07-Cres_04_overlap_removed_15,A je bilo ča za mesopusno vrime?,339.817105,342.209639,4,1,1
4,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Mi, ja znan da kad san ja krenula va školu da ...",6.517170,12.770724,1,2,1
...,...,...,...,...,...,...,...
4503,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Tako da @ sad @ će njoj bit malo legje da se m...,581.117400,586.880631,162,2,1
4504,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,"Mlada, a toliko @ va poslu @ ni dobro.",587.446607,590.414025,163,2,1
4505,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Rabela bi kakovoga mladića nać i oženit se i @...,590.414025,596.167381,164,2,1
4506,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,To @ ni od toga koristi baš.,596.167381,598.298191,165,2,1


In [46]:
a = [1,1,0]
a.count(1)

2

In [48]:
# get tier word percentage
tier_type = []

for file_name in file_names:
    #print (file_name)
    df_sub = df[df['file'] == file_name]# choose subset with file name
    tiers = list(set(df_sub['tier'].tolist()))
    for tier in tiers:
        df_sub_curr = df_sub[df_sub['tier'] == tier]
        sent_type_list = df_sub_curr['sent'].tolist()
        tier_sent_percent = sent_type_list.count(1)/len(sent_type_list)
        print (file_name, tier, tier_sent_percent, len(sent_type_list))
        if tier_sent_percent > 0.5: # current layer is a sentence layer
            tier_type = tier_type + [1]*len(df_sub_curr)
        else:
            tier_type = tier_type + [0]*len(df_sub_curr)

ckm004-2022-07-07-Cres_04_overlap_removed_15 1 1.0 4
ckm004-2022-07-07-Cres_04_overlap_removed_15 2 0.9795918367346939 147
ckm002-2022-01-04-Crikvenica_07_overlap_removed_15 1 0.9411764705882353 17
ckm002-2022-01-04-Crikvenica_07_overlap_removed_15 2 0.6464088397790055 181
ckm002-2022-01-04-Crikvenica_02_overlap_removed_15 1 1.0 13
ckm002-2022-01-04-Crikvenica_02_overlap_removed_15 2 0.6395939086294417 197
ckm001-2022-01-16-Trviž_02_overlap_removed_15 1 0.8888888888888888 9
ckm002-2022-01-04-Crikvenica_04_overlap_removed_15 1 0.9090909090909091 11
ckm002-2022-01-04-Crikvenica_04_overlap_removed_15 2 0.6188118811881188 202
ckm006-2023-05-28-Kostrena_05_overlap_removed_15 1 0.9090909090909091 11
ckm006-2023-05-28-Kostrena_05_overlap_removed_15 2 0.6334661354581673 251
ckm005-2022-11-26-Vrbnik_03_overlap_removed_15 1 0.8888888888888888 27
ckm005-2022-11-26-Vrbnik_03_overlap_removed_15 2 0.9383561643835616 146
ckm004-2022-07-07-Cres_02_overlap_removed_15 1 1.0 2
ckm004-2022-07-07-Cres_02_

In [49]:
len(tier_type)

4508

In [50]:
df['tier_sent'] = tier_type
df # tier sent marked, and checked correct

,file,text,xmin,xmax,interval,tier,sent,tier_sent
0,ckm004-2022-07-07-Cres_04_overlap_removed_15,Na Grobniku se jako dobro čuva govor,0.000000,4.441976,1,1,1,1
1,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Va škole se puno čuva, i Katedra je jaka.",117.091133,120.544704,2,1,1,1
2,ckm004-2022-07-07-Cres_04_overlap_removed_15,"A puno se na Grobnik radilo z drvom, ča ne?",202.971944,206.757870,3,1,1,1
3,ckm004-2022-07-07-Cres_04_overlap_removed_15,A je bilo ča za mesopusno vrime?,339.817105,342.209639,4,1,1,1
4,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Mi, ja znan da kad san ja krenula va školu da ...",6.517170,12.770724,1,2,1,1
...,...,...,...,...,...,...,...,...
4503,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Tako da @ sad @ će njoj bit malo legje da se m...,581.117400,586.880631,162,2,1,1
4504,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,"Mlada, a toliko @ va poslu @ ni dobro.",587.446607,590.414025,163,2,1,1
4505,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Rabela bi kakovoga mladića nać i oženit se i @...,590.414025,596.167381,164,2,1,1
4506,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,To @ ni od toga koristi baš.,596.167381,598.298191,165,2,1,1


In [51]:
df['chunk_len'] = df.apply(lambda x: x.xmax - x.xmin, axis = 1)
df

,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len
0,ckm004-2022-07-07-Cres_04_overlap_removed_15,Na Grobniku se jako dobro čuva govor,0.000000,4.441976,1,1,1,1,4.441976
1,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Va škole se puno čuva, i Katedra je jaka.",117.091133,120.544704,2,1,1,1,3.453571
2,ckm004-2022-07-07-Cres_04_overlap_removed_15,"A puno se na Grobnik radilo z drvom, ča ne?",202.971944,206.757870,3,1,1,1,3.785926
3,ckm004-2022-07-07-Cres_04_overlap_removed_15,A je bilo ča za mesopusno vrime?,339.817105,342.209639,4,1,1,1,2.392535
4,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Mi, ja znan da kad san ja krenula va školu da ...",6.517170,12.770724,1,2,1,1,6.253554
...,...,...,...,...,...,...,...,...,...
4503,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Tako da @ sad @ će njoj bit malo legje da se m...,581.117400,586.880631,162,2,1,1,5.763231
4504,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,"Mlada, a toliko @ va poslu @ ni dobro.",587.446607,590.414025,163,2,1,1,2.967418
4505,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Rabela bi kakovoga mladića nać i oženit se i @...,590.414025,596.167381,164,2,1,1,5.753356
4506,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,To @ ni od toga koristi baš.,596.167381,598.298191,165,2,1,1,2.130811


In [52]:
df.to_csv(save_path + '/cr_text_info_overlap_removed.csv')

In [53]:
# get sentence segmentation 

df_sent = df[df['tier_sent'] == 1] # ignore the word tiers
df_sent.reset_index(inplace = True)
df_sent

,index,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len
0,0,ckm004-2022-07-07-Cres_04_overlap_removed_15,Na Grobniku se jako dobro čuva govor,0.000000,4.441976,1,1,1,1,4.441976
1,1,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Va škole se puno čuva, i Katedra je jaka.",117.091133,120.544704,2,1,1,1,3.453571
2,2,ckm004-2022-07-07-Cres_04_overlap_removed_15,"A puno se na Grobnik radilo z drvom, ča ne?",202.971944,206.757870,3,1,1,1,3.785926
3,3,ckm004-2022-07-07-Cres_04_overlap_removed_15,A je bilo ča za mesopusno vrime?,339.817105,342.209639,4,1,1,1,2.392535
4,4,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Mi, ja znan da kad san ja krenula va školu da ...",6.517170,12.770724,1,2,1,1,6.253554
...,...,...,...,...,...,...,...,...,...,...
4478,4503,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Tako da @ sad @ će njoj bit malo legje da se m...,581.117400,586.880631,162,2,1,1,5.763231
4479,4504,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,"Mlada, a toliko @ va poslu @ ni dobro.",587.446607,590.414025,163,2,1,1,2.967418
4480,4505,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Rabela bi kakovoga mladića nać i oženit se i @...,590.414025,596.167381,164,2,1,1,5.753356
4481,4506,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,To @ ni od toga koristi baš.,596.167381,598.298191,165,2,1,1,2.130811


In [49]:
non_sent_list = ['((breath))',
                '((laughs))',
                '((breaths)',
                '@',
                '((coughs))',
                '((click))',
                '(())',
                '#',
                'x',
                'Mhm.',
                'Mhm, mhm.',
                '((hhh))',
                'Mhm',
                 '((noise))',                            
'((overlapping, unintelligible))',      
'((breath, smack))',  
'((breath, coughs))',                   
'((breath)) ((click))',                 
'(x)',            
'((machine sounds))',                   
'((breath, noise))',                                                
'((breath, click))' 
                ]

In [16]:
df_sent['text'].value_counts()[:40]

((breath))                         924
Mhm.                               137
((laughs))                          59
((breaths))                         57
Da.                                 48
@                                   47
Dobro.                              15
((coughs))                          14
((click))                           12
#                                   10
(())                                10
Ja.                                  9
x                                    9
Mhm, mhm.                            8
I tako.                              7
((hhh))                              7
Mhm                                  6
((noise))                            6
Tako da-                             6
Tako.                                6
Ne.                                  5
Aha.                                 5
((overlapping, unintelligible))      5
((breath, smack))                    5
((breath)) ((click))                 4
((breath, coughs))       

In [16]:
import numpy as np
np.max(df_sent_2['chunk_len'])

NameError: name 'df_sent_2' is not defined

In [22]:
len(xmin_list)

9812

In [54]:
df_sent['file_tier'] = df_sent.apply(lambda x: x.file + '_' + str(x.tier), axis = 1)
df_sent

<ipython-input-54-c2c3ea93b3ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sent['file_tier'] = df_sent.apply(lambda x: x.file + '_' + str(x.tier), axis = 1)


,index,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len,file_tier
0,0,ckm004-2022-07-07-Cres_04_overlap_removed_15,Na Grobniku se jako dobro čuva govor,0.000000,4.441976,1,1,1,1,4.441976,ckm004-2022-07-07-Cres_04_overlap_removed_15_1
1,1,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Va škole se puno čuva, i Katedra je jaka.",117.091133,120.544704,2,1,1,1,3.453571,ckm004-2022-07-07-Cres_04_overlap_removed_15_1
2,2,ckm004-2022-07-07-Cres_04_overlap_removed_15,"A puno se na Grobnik radilo z drvom, ča ne?",202.971944,206.757870,3,1,1,1,3.785926,ckm004-2022-07-07-Cres_04_overlap_removed_15_1
3,3,ckm004-2022-07-07-Cres_04_overlap_removed_15,A je bilo ča za mesopusno vrime?,339.817105,342.209639,4,1,1,1,2.392535,ckm004-2022-07-07-Cres_04_overlap_removed_15_1
4,4,ckm004-2022-07-07-Cres_04_overlap_removed_15,"Mi, ja znan da kad san ja krenula va školu da ...",6.517170,12.770724,1,2,1,1,6.253554,ckm004-2022-07-07-Cres_04_overlap_removed_15_2
...,...,...,...,...,...,...,...,...,...,...,...
4478,4503,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Tako da @ sad @ će njoj bit malo legje da se m...,581.117400,586.880631,162,2,1,1,5.763231,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2
4479,4504,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,"Mlada, a toliko @ va poslu @ ni dobro.",587.446607,590.414025,163,2,1,1,2.967418,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2
4480,4505,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,Rabela bi kakovoga mladića nać i oženit se i @...,590.414025,596.167381,164,2,1,1,5.753356,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2
4481,4506,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15,To @ ni od toga koristi baš.,596.167381,598.298191,165,2,1,1,2.130811,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2


In [18]:
df_sent_2 = df_sent[df_sent['text'].isin(non_sent_list) == False]
#df_sent_2 = df_sent

df_sent_2.reset_index(inplace = True, drop = True)
df_sent_2

NameError: name 'non_sent_list' is not defined

# clean the text (())

In [51]:
symbol_list = ['((breath))',
                '((laughs))',
                '((breaths)',
               '((breaths))',
                '((coughs))',
                '((click))',
                '(())',
                '#',
                 '((noise))',                            
'((overlapping, unintelligible))',      
'((breath, smack))',  
'((breath, coughs))',                   
'((breath)) ((click))',                 
'(x)',            
'((machine sounds))',                   
'((breath, noise))',                                               
'((breath, click))' 
                ]

In [28]:
def string_clean(text):
    for symbol in symbol_list:
        text = text.replace(symbol, '@')
    
    return text

text = "xdjii ((breath)) jisjdioj"
string_clean(text)

'xdjii @ jisjdioj'

In [52]:
#df_sent_2['text_clean'] = df_sent_2['text'].apply(lambda x: string_clean(x))

<ipython-input-52-1a273dae82a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sent_2['text_clean'] = df_sent_2['text'].apply(lambda x: string_clean(x))


In [55]:
l1 = df_sent['file_tier'].tolist()

file_tier_list = sorted(set(l1), key=l1.index)

In [56]:
file_tier_list

['ckm004-2022-07-07-Cres_04_overlap_removed_15_1',
 'ckm004-2022-07-07-Cres_04_overlap_removed_15_2',
 'ckm002-2022-01-04-Crikvenica_07_overlap_removed_15_1',
 'ckm002-2022-01-04-Crikvenica_07_overlap_removed_15_2',
 'ckm002-2022-01-04-Crikvenica_02_overlap_removed_15_1',
 'ckm002-2022-01-04-Crikvenica_02_overlap_removed_15_2',
 'ckm001-2022-01-16-Trviž_02_overlap_removed_15_1',
 'ckm002-2022-01-04-Crikvenica_04_overlap_removed_15_1',
 'ckm002-2022-01-04-Crikvenica_04_overlap_removed_15_2',
 'ckm006-2023-05-28-Kostrena_05_overlap_removed_15_1',
 'ckm006-2023-05-28-Kostrena_05_overlap_removed_15_2',
 'ckm005-2022-11-26-Vrbnik_03_overlap_removed_15_1',
 'ckm005-2022-11-26-Vrbnik_03_overlap_removed_15_2',
 'ckm004-2022-07-07-Cres_02_overlap_removed_15_1',
 'ckm004-2022-07-07-Cres_02_overlap_removed_15_2',
 'ckm006-2023-05-28-Kostrena_02_overlap_removed_15_1',
 'ckm006-2023-05-28-Kostrena_02_overlap_removed_15_2',
 'ckm006-2023-05-28-Kostrena_03_overlap_removed_15_1',
 'ckm006-2023-05-28-

# read the textgrid file and get reasonable chunk time points


In [123]:
def locate_end(xmin_list, xmax_list, i):
    start_curr = xmin_list[i]
    if xmax_list[-1] - start_curr <= 20: # if the end of max < 15, return the last xmax
        return xmax_list[-1], len(xmin_list) + 1
    if xmin_list[i + 1] - start_curr > 30: # if next start leaves too long of blank
        return start_curr + 20, i + 1
    while i < len(xmin_list):
        end_curr = xmax_list[i]
        #print ('----', end_curr, i)
        if end_curr - start_curr >= 20:
            if end_curr - start_curr <= 25:
                return end_curr, i + 1
            else:
                return start_curr + (25 - (xmax_list[i-1] - start_curr)), i 
        else:
            i = i + 1
            continue
    

In [19]:
def locate_end_small(xmin_list, xmax_list, i, chunk_size):
    
    start_curr = xmin_list[i]
    end_curr = xmax_list[i]

    if xmax_list[-1] - start_curr <= chunk_size: # if the end of max < 15, return the last xmax
        return xmax_list[-1], len(xmin_list) + 1

    if end_curr - start_curr >= chunk_size:
        return end_curr, i + 1
    
    while i < len(xmin_list):
        end_curr = xmax_list[i]
        #print ('----', start_curr, end_curr, i)
        if end_curr - start_curr >= chunk_size:
            
            if xmin_list[i] - start_curr >= chunk_size: # current start - last end > 10 blank, sep the blank and extend the last one
                return start_curr + chunk_size - 2,i
            else:
                return end_curr, i + 1
      
        else:
            i = i + 1
            continue

In [67]:
# chunk size 15 - 20 seconds
chunk_size = 30

start = []
end = []
start_id = []
end_id = []
file_info = []
chunk_name = []

for file_name in file_tier_list:
#for file_name in [file_names[0]]:
    print (file_name)

    df_curr = df_sent[df_sent['file_tier'] == file_name]
    xmin_list = df_curr['xmin'].tolist()
    xmax_list = df_curr['xmax'].tolist()
    
    i = 0
    j = 0 # counting the file chunk number
    while i < len(xmin_list):
        start_curr = xmin_list[i]
        i_curr = i
        # locate the one that > 15 and  < 25 in endlist
        end_curr = locate_end_small(xmin_list, xmax_list, i, chunk_size)[0] # change locate end function
        i = locate_end_small(xmin_list, xmax_list, i, chunk_size)[1]
        print (start_curr, end_curr, i_curr, i)
        
        j = j + 1 # chunk number update
        
        start.append(start_curr)
        end.append(end_curr)
        start_id.append(i_curr)
        end_id.append(i)
        file_info.append(file_name)
        chunk_name.append(file_name + '_' + str(j))

ckm004-2022-07-07-Cres_04_overlap_removed_15_1
0.0 28.0 0 1
117.0911330744798 145.09113307447979 1 2
202.97194351490424 230.97194351490424 2 3
339.81710468448733 342.2096392823917 3 5
ckm004-2022-07-07-Cres_04_overlap_removed_15_2
6.517169572479057 34.517169572479055 0 7
36.79513951456555 64.79513951456555 7 16
67.00884641085854 103.83475499304383 16 23
104.37500474095772 138.63260215460338 23 31
139.0699471886289 171.7280714036943 31 41
172.23230450174725 200.23230450174725 41 49
210.56131700908634 241.2221077630512 49 58
241.96107985053752 269.96107985053754 58 67
272.264348451061 305.0253699897405 67 76
305.4832959665437 336.5059229428933 76 83
336.8683182032949 368.1934033998563 83 91
368.1934033998563 401.12755716400267 91 100
401.12755716400267 429.12755716400267 100 108
431.393139390888 463.0676850312943 108 116
463.7314204358742 496.91936047282235 116 126
497.2560295547522 528.5044878636389 126 134
528.9567169570053 560.6804861925629 134 142
561.1023955195051 577.231212713468 1

0.2769659062678104 28.27696590626781 0 6
176.95910295343342 204.95910295343342 6 7
306.50594115402095 334.50594115402095 7 8
368.968798167985 396.968798167985 8 9
406.3373215699841 434.3373215699841 9 10
439.4628238874798 467.4628238874798 10 14
510.4151911612844 538.4151911612844 14 17
576.9063595638864 577.2562355911068 17 19
ckm005-2022-11-26-Vrbnik_01_overlap_removed_15_2
16.96090221056265 48.20282526898012 0 7
49.146911700931895 82.53829654709524 7 18
83.19688671598075 115.81537552882483 18 30
116.52723125287767 146.93256619805555 30 37
147.4007826462476 175.4007826462476 37 47
184.57810117254135 214.98274101606958 47 54
214.98274101606958 248.53112728630322 54 61
248.53112728630322 283.65938785628185 61 68
283.9835377050302 314.12910998302993 68 74
314.12910998302993 342.12910998302993 74 79
344.2763519884722 377.2906009548615 79 88
377.79914813359477 405.79914813359477 88 96
409.72956556668527 437.72956556668527 96 106
450.67653607617757 483.5791235313701 106 116
483.57912353137

In [68]:
df_chunk = pd.DataFrame()

df_chunk['chunk_name'] = chunk_name
df_chunk['file_tier'] = file_info
df_chunk['start'] = start
df_chunk['end'] = end
df_chunk['start_id'] = start_id
df_chunk['end_id'] = end_id
df_chunk

,chunk_name,file_tier,start,end,start_id,end_id
0,ckm004-2022-07-07-Cres_04_overlap_removed_15_1_1,ckm004-2022-07-07-Cres_04_overlap_removed_15_1,0.000000,28.000000,0,1
1,ckm004-2022-07-07-Cres_04_overlap_removed_15_1_2,ckm004-2022-07-07-Cres_04_overlap_removed_15_1,117.091133,145.091133,1,2
2,ckm004-2022-07-07-Cres_04_overlap_removed_15_1_3,ckm004-2022-07-07-Cres_04_overlap_removed_15_1,202.971944,230.971944,2,3
3,ckm004-2022-07-07-Cres_04_overlap_removed_15_1_4,ckm004-2022-07-07-Cres_04_overlap_removed_15_1,339.817105,342.209639,3,5
4,ckm004-2022-07-07-Cres_04_overlap_removed_15_2_1,ckm004-2022-07-07-Cres_04_overlap_removed_15_2,6.517170,34.517170,0,7
...,...,...,...,...,...,...
572,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15...,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2,446.291125,476.682974,121,130
573,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15...,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2,476.682974,507.598526,130,139
574,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15...,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2,507.794045,538.140146,139,150
575,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15...,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2,538.140146,569.798698,150,159


In [69]:
# get text for the chunks from original un-chunked data

def get_chunk_text(start_id, end_id, file_tier):
    df_curr = df_sent[df_sent['file_tier'] == file_tier] 
    text_list = df_curr['text'].tolist() # change to clean
    text_chunks = text_list[start_id : end_id]
    text = ' '.join(text_chunks)
    return text

get_chunk_text(0, 2, 'ckm002-2022-01-04-Crikvenica_05_1')

''

In [70]:
df_chunk['text'] = df_chunk.apply(lambda x: get_chunk_text(x.start_id, x.end_id, x.file_tier), axis = 1)
df_chunk

,chunk_name,file_tier,start,end,start_id,end_id,text
0,ckm004-2022-07-07-Cres_04_overlap_removed_15_1_1,ckm004-2022-07-07-Cres_04_overlap_removed_15_1,0.000000,28.000000,0,1,Na Grobniku se jako dobro čuva govor
1,ckm004-2022-07-07-Cres_04_overlap_removed_15_1_2,ckm004-2022-07-07-Cres_04_overlap_removed_15_1,117.091133,145.091133,1,2,"Va škole se puno čuva, i Katedra je jaka."
2,ckm004-2022-07-07-Cres_04_overlap_removed_15_1_3,ckm004-2022-07-07-Cres_04_overlap_removed_15_1,202.971944,230.971944,2,3,"A puno se na Grobnik radilo z drvom, ča ne?"
3,ckm004-2022-07-07-Cres_04_overlap_removed_15_1_4,ckm004-2022-07-07-Cres_04_overlap_removed_15_1,339.817105,342.209639,3,5,A je bilo ča za mesopusno vrime?
4,ckm004-2022-07-07-Cres_04_overlap_removed_15_2_1,ckm004-2022-07-07-Cres_04_overlap_removed_15_2,6.517170,34.517170,0,7,"Mi, ja znan da kad san ja krenula va školu da ..."
...,...,...,...,...,...,...,...
572,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15...,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2,446.291125,476.682974,121,130,Od mala su se učili i morali su naučit. Makar ...
573,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15...,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2,476.682974,507.598526,130,139,i u nas već dela @ ne znan @ kada se fini evo ...
574,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15...,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2,507.794045,538.140146,139,150,i ni šel deje @ nego je videl da je potriba do...
575,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15...,ckm005-2022-11-26-Vrbnik_04_overlap_removed_15_2,538.140146,569.798698,150,159,((hhh)) ((hhh)) grafički dizajn @ i dela va je...


In [71]:
df_chunk.to_csv(save_path + '/cr_chunk_text_info_overlap_removed_' + str(chunk_size) +'.csv')

# get audio chunks

In [142]:
# get audio files according tp df_chunk info

save_chunk_path = save_path + '/cr_audio_chunk/'

In [144]:
# read in the audio, for each row in df_chunk, get the chunked audio and save
from pydub import AudioSegment

def get_audio_chunk(start, end, chunk_name, file_tier):
    audio_dir = folder_path +'/'+ file_tier[:-2] + '.wav'
    t1 = start * 1000 # s to ms
    t2 = end * 1000
    newAudio = AudioSegment.from_wav(audio_dir)[t1:t2]
    newAudio.export(save_chunk_path + chunk_name + '.wav', format="wav")
    return chunk_name + '.wav'

get_audio_chunk(0, 10, 'ckm002-2022-01-04-Crikvenica_05_1_1', 'ckm002-2022-01-04-Crikvenica_05_1')    

'ckm002-2022-01-04-Crikvenica_05_1_1.wav'

In [145]:
df_chunk['audio_name'] = df_chunk.apply(lambda x: get_audio_chunk(x.start, x.end, x.chunk_name, x.file_tier), axis = 1)
df_chunk

,chunk_name,file_tier,start,end,start_id,end_id,text,audio_name
0,ckm002-2022-01-04-Crikvenica_05_1_1,ckm002-2022-01-04-Crikvenica_05_1,31.204190,51.204190,0,1,Ste voleli školu?,ckm002-2022-01-04-Crikvenica_05_1_1.wav
1,ckm002-2022-01-04-Crikvenica_05_1_2,ckm002-2022-01-04-Crikvenica_05_1,65.324119,79.870934,1,3,Dal' van je baš neka njegova anegdota ili neč ...,ckm002-2022-01-04-Crikvenica_05_1_2.wav
2,ckm002-2022-01-04-Crikvenica_05_1_3,ckm002-2022-01-04-Crikvenica_05_1,172.354525,191.136708,3,5,"A kad ste malo divojke porasle, kako je to neg...",ckm002-2022-01-04-Crikvenica_05_1_3.wav
3,ckm002-2022-01-04-Crikvenica_05_1_4,ckm002-2022-01-04-Crikvenica_05_1,218.237587,237.273134,5,7,"A tu se trebalo i dobro snać, jer tad ni bilo ...",ckm002-2022-01-04-Crikvenica_05_1_4.wav
4,ckm002-2022-01-04-Crikvenica_05_1_5,ckm002-2022-01-04-Crikvenica_05_1,282.554819,302.554819,7,8,Ste voleli tancat?,ckm002-2022-01-04-Crikvenica_05_1_5.wav
...,...,...,...,...,...,...,...,...
876,ckm005-2022-11-26-Vrbnik_06_2_22,ckm005-2022-11-26-Vrbnik_06_2,481.870624,504.014365,117,124,"((hhh)) svi za njin. Onda bi bil, blo po tancu...",ckm005-2022-11-26-Vrbnik_06_2_22.wav
877,ckm005-2022-11-26-Vrbnik_06_2_23,ckm005-2022-11-26-Vrbnik_06_2,504.014365,524.812835,124,130,A izmedžu se @ kanta @ da se zabavi @ malo sop...,ckm005-2022-11-26-Vrbnik_06_2_23.wav
878,ckm005-2022-11-26-Vrbnik_06_2_24,ckm005-2022-11-26-Vrbnik_06_2,525.281052,537.900897,130,134,malo @ pokazat na placi da judi vide to bi bil...,ckm005-2022-11-26-Vrbnik_06_2_24.wav
879,ckm005-2022-11-26-Vrbnik_06_2_25,ckm005-2022-11-26-Vrbnik_06_2,538.139161,558.497226,134,137,"Ni @ valjda mladi il ki ne vole sopeli, ne vol...",ckm005-2022-11-26-Vrbnik_06_2_25.wav


In [ ]:
# goal: data info dataframe, cols are audio address, audio text, audio original time info and file info

